In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

np.random.seed(42)
tf.random.set_seed(42)

file_path = '/workspaces/A01567178-EDA/data/prueba modelo cuadrante negocio.xlsx'

# cargar datos
df = pd.read_excel(file_path)

In [5]:
df.columns = ['Cuadrante', 'Mes_N', 'Robos_N', 'Robos_N_1', 'Robos_N_2']

In [6]:
df['Index'] = df['Cuadrante'].astype(str) + '_' + df['Mes_N'].astype(str)

In [7]:
# Las variables independientes (X) son el Mes_N, Robos_N_1, Robos_N_2
X_data = df[['Mes_N', 'Robos_N_1', 'Robos_N_2']].copy()
# La variable dependiente (Y) es Robos_N, pivotada por Cuadrante
Y_data = df.pivot(index='Mes_N', columns='Cuadrante', values='Robos_N')

In [8]:
X = X_data.groupby('Mes_N').mean().reset_index()

In [9]:
Y = Y_data.values

In [10]:
features = ['Mes_N', 'Robos_N_1', 'Robos_N_2']
X = df[features].values

In [11]:
# --- 3. PREPARACIÓN PARA ENTRENAMIENTO MULTIPLE ---

# Inicializar un diccionario para almacenar los modelos entrenados
models = {}
# Inicializar un diccionario para almacenar los escaladores (importante para la normalización)
scalers = {}
# Obtener la lista única de cuadrantes
cuadrantes_unicos = df['Cuadrante'].unique()

In [12]:
# 3.1 Normalización (Escalado)
# Es crucial en Redes Neuronales. Usaremos el mismo escalador para todos los cuadrantes.
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
# Escalamos la variable objetivo (Y) para una mejor convergencia del modelo
scaler_Y = StandardScaler()
Y_scaled = scaler_Y.fit_transform(df['Robos_N'].values.reshape(-1, 1))

In [13]:
# Re-ensamblar los datos escalados para la división por cuadrante
df_scaled = pd.DataFrame(X_scaled, columns=features)
df_scaled['Robos_N'] = Y_scaled
df_scaled['Cuadrante'] = df['Cuadrante'].values

In [14]:
def build_keras_model(input_shape):
    """Define la arquitectura del modelo DNN (Red Neuronal Densa) simple."""
    model = Sequential([
        # Capa de entrada con 3 features (Mes_N, Robos_N_1, Robos_N_2)
        Input(shape=(input_shape,)),
        # Primera capa oculta
        Dense(32, activation='relu'),
        # Regularización para evitar overfitting
        Dense(16, activation='relu'),
        # Capa de salida. Una neurona para la predicción de robos.
        # 'linear' porque es un problema de regresión.
        Dense(1, activation='linear')
    ])

    # Compilación del modelo
    model.compile(
        optimizer='adam',
        # MSE o MAE son ideales para regresión de conteos
        loss='mean_absolute_error',
        metrics=['mae', 'mse']
    )
    return model

In [15]:
# --- 5. ENTRENAMIENTO POR CUADRANTE ---

for cuadrante_id in cuadrantes_unicos:
    print(f"\nEntrenando modelo para Cuadrante {cuadrante_id}...")

    # 5.1 Filtrar datos para el cuadrante actual
    df_cuadrante = df_scaled[df_scaled['Cuadrante'] == cuadrante_id]

    X_cuadrante = df_cuadrante[features].values
    Y_cuadrante = df_cuadrante['Robos_N'].values.reshape(-1, 1) # Asegurar formato 2D

    # 5.2 División de datos (Train/Test)
    # Usaremos los datos más antiguos para entrenar (80%) y los más recientes para probar (20%)
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_cuadrante, Y_cuadrante, test_size=0.2, shuffle=False # 'shuffle=False' es CRÍTICO en series temporales
    )

    # 5.3 Construir y entrenar el modelo
    model = build_keras_model(X_train.shape[1])

    # Early Stopping para detener el entrenamiento si la métrica no mejora (previene overfitting)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, Y_train,
        epochs=100, # Número máximo de pasadas
        batch_size=4, # Pequeño tamaño de lote dado el pequeño dataset por cuadrante (93 filas)
        validation_data=(X_test, Y_test),
        callbacks=[early_stop],
        verbose=0 # No mostrar el progreso del entrenamiento
    )

    # 5.4 Evaluación (Opcional)
    loss, mae, mse = model.evaluate(X_test, Y_test, verbose=0)
    # Desescalar MAE para que sea interpretable en número de robos
    mae_unscaled = scaler_Y.inverse_transform(np.array([[mae]]))
    print(f"-> MAE (Error Absoluto Medio) en datos de prueba: {mae_unscaled[0][0]:.2f} robos")

    # 5.5 Guardar el modelo entrenado
    models[cuadrante_id] = model


Entrenando modelo para Cuadrante 1...


2025-11-07 03:12:30.574010: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


-> MAE (Error Absoluto Medio) en datos de prueba: 15.98 robos

Entrenando modelo para Cuadrante 2...
-> MAE (Error Absoluto Medio) en datos de prueba: 15.68 robos

Entrenando modelo para Cuadrante 3...
-> MAE (Error Absoluto Medio) en datos de prueba: 15.76 robos

Entrenando modelo para Cuadrante 4...
-> MAE (Error Absoluto Medio) en datos de prueba: 16.52 robos

Entrenando modelo para Cuadrante 5...
-> MAE (Error Absoluto Medio) en datos de prueba: 16.02 robos

Entrenando modelo para Cuadrante 6...
-> MAE (Error Absoluto Medio) en datos de prueba: 23.86 robos

Entrenando modelo para Cuadrante 7...
-> MAE (Error Absoluto Medio) en datos de prueba: 28.94 robos

Entrenando modelo para Cuadrante 8...
-> MAE (Error Absoluto Medio) en datos de prueba: 21.64 robos

Entrenando modelo para Cuadrante 9...
-> MAE (Error Absoluto Medio) en datos de prueba: 17.06 robos

Entrenando modelo para Cuadrante 10...
-> MAE (Error Absoluto Medio) en datos de prueba: 18.55 robos

Entrenando modelo para Cuad

In [16]:
print("\n--- ¡ENTRENAMIENTO COMPLETO! ---")
print(f"Se han entrenado y guardado {len(models)} modelos de Red Neuronal, uno para cada cuadrante.")


--- ¡ENTRENAMIENTO COMPLETO! ---
Se han entrenado y guardado 78 modelos de Red Neuronal, uno para cada cuadrante.


In [17]:
# --- 6. FUNCIÓN DE PREDICCIÓN (Ejemplo) ---

def predict_next_month_robos(models, scaler_X, scaler_Y, current_month, robos_n_1, robos_n_2, cuadrantes):
    """
    Predice el total de robos para el próximo mes (current_month + 1) en todos los cuadrantes.

    :param current_month: El número de mes para el cual quieres predecir (1 a 12).
    :param robos_n_1: Un array de 78 elementos con los robos del mes anterior (N-1).
    :param robos_n_2: Un array de 78 elementos con los robos del mes anterior (N-2).
    """

    predictions = {}

    # Asume que robos_n_1 y robos_n_2 son listas o arrays ya ordenados por el id de cuadrante

    for i, cuadrante_id in enumerate(cuadrantes):
        model = models[cuadrante_id]

        # 1. Crear el vector de entrada (features) para el cuadrante
        # Input: [Mes_N, Robos_N_1, Robos_N_2]
        # NOTA: Usamos (current_month + 1) si queremos predecir el siguiente mes

        # Para el propósito de la predicción, N será (el mes en el que estamos + 1).
        # Para este ejemplo, solo usaremos los valores de entrada proporcionados:

        input_data = np.array([[current_month, robos_n_1[i], robos_n_2[i]]])

        # 2. Escalar el input usando el escalador de entrenamiento
        input_scaled = scaler_X.transform(input_data)

        # 3. Realizar la predicción
        prediction_scaled = model.predict(input_scaled, verbose=0)

        # 4. Desescalar la predicción para obtener el número real de robos
        prediction_unscaled = scaler_Y.inverse_transform(prediction_scaled)

        predictions[cuadrante_id] = round(prediction_unscaled[0][0])

    return predictions

In [18]:
# --- 7. SIMULACIÓN DE PREDICCIÓN ---

print("\n--- PREDICCIÓN SIMULADA (Próximo Mes) ---")

# Tomar los últimos valores de N-1 y N-2 de la base de datos como ejemplo de entrada
# OTRA VEZ: Usaremos los valores del Mes 12 (el último) para simular la entrada del próximo mes.

# Filtrar solo el último Mes_N registrado (Mes 12 de cada cuadrante en el dataset)
last_month_data = df[df['Mes_N'] == df['Mes_N'].max()]

# Ordenar por cuadrante para asegurar la alineación
last_month_data = last_month_data.sort_values(by='Cuadrante').reset_index(drop=True)

# Las entradas para el Mes 1 (del nuevo ciclo anual) serían los robos N-1 y N-2 de ese último mes
# En un escenario real, 'current_month' sería 1 (para enero del nuevo año)

# NOTA: Los nombres de las columnas en tu base son ligeramente confusos.
# Asumo que las entradas para el Mes N (el que predices) son:
# - Mes N: 1 (el mes que vas a predecir)
# - Robos N-1: Los robos del último mes real (Mes 12)
# - Robos N-2: Los robos del penúltimo mes real (Mes 11)

# Para simular, tomaremos los valores del Mes 12 como si fueran los inputs N-1 (Robos_N) y N-2 (Robos_N_1)
# para el siguiente mes de predicción.

# Supongamos que queremos predecir el Mes 1 (del siguiente ciclo)
mes_a_predecir = 1
# Los robos N-1 son los robos del último Mes_N de tu dataset (Mes 12)
robos_n_menos_1 = last_month_data['Robos_N'].tolist()
# Los robos N-2 son los robos del penúltimo Mes_N (es decir, Robos_N_1 del último mes en el dataset)
robos_n_menos_2 = last_month_data['Robos_N_1'].tolist()

# Lista de IDs de cuadrantes ordenados para la predicción
cuadrantes_ordenados = last_month_data['Cuadrante'].tolist()

# Ejecutar la predicción
predicted_robos = predict_next_month_robos(
    models,
    scaler_X,
    scaler_Y,
    mes_a_predecir,
    robos_n_menos_1,
    robos_n_menos_2,
    cuadrantes_ordenados
)


--- PREDICCIÓN SIMULADA (Próximo Mes) ---


In [19]:
# 7.1 Mostrar y Rankear resultados
results_df = pd.DataFrame(list(predicted_robos.items()), columns=['Cuadrante', 'Robos_Predichos_Mes_N'])
results_df['Cuadrante'] = results_df['Cuadrante'].astype(int)

# Ordenar de mayor a menor incidencia delictiva
ranking = results_df.sort_values(by='Robos_Predichos_Mes_N', ascending=False)

print("\nTOP 5 Cuadrantes con Mayor Incidencia Delictiva (Predicción Mes N):")
print(ranking.head())


TOP 5 Cuadrantes con Mayor Incidencia Delictiva (Predicción Mes N):
    Cuadrante  Robos_Predichos_Mes_N
54         55                     50
12         13                     49
24         25                     49
7           8                     42
30         31                     39


In [20]:
print("\nResumen de la Predicción (Primeros 10 Cuadrantes):")
print(ranking.head(10).to_string(index=False))


Resumen de la Predicción (Primeros 10 Cuadrantes):
 Cuadrante  Robos_Predichos_Mes_N
        55                     50
        13                     49
        25                     49
         8                     42
        31                     39
        37                     39
        77                     33
        22                     31
        69                     30
        70                     30


In [21]:
# predecir para mes 12 cuadrante 78


# Task
Predict the number of robberies for month 12 in quadrant 78 using the trained model and display the result.

## Preparar Datos de Entrada

### Subtask:
Obtener los valores de 'Robos_N' y 'Robos_N_1' de los dos meses anteriores al mes 12 para el cuadrante 78. Estos se usarán como 'Robos_N_1' y 'Robos_N_2' para la predicción del mes 12.


In [22]:
print("\n--- Preparando datos de entrada para la predicción ---")

# 1. Filtra el DataFrame `df` para obtener la fila correspondiente al 'Cuadrante' 78 y 'Mes_N' 11.
data_for_quadrant_78_month_11 = df[(df['Cuadrante'] == 78) & (df['Mes_N'] == 11)]

# Verificar que se encontró la fila
if not data_for_quadrant_78_month_11.empty:
    # 2. Extrae el valor de la columna 'Robos_N' de la fila filtrada
    robos_n_1_for_pred = data_for_quadrant_78_month_11['Robos_N'].iloc[0]
    # 3. Extrae el valor de la columna 'Robos_N_1' de la fila filtrada
    robos_n_2_for_pred = data_for_quadrant_78_month_11['Robos_N_1'].iloc[0]
else:
    # Si no se encuentra la fila, se asignan valores predeterminados o se maneja el error
    print("Advertencia: No se encontraron datos para Cuadrante 78 y Mes_N 11. Usando valores predeterminados (0).")
    robos_n_1_for_pred = 0
    robos_n_2_for_pred = 0

# 4. Establece el `mes_a_predecir` como 12.
mes_a_predecir_specific = 12

print(f"Valores obtenidos: Mes a predecir: {mes_a_predecir_specific}, Robos N-1: {robos_n_1_for_pred}, Robos N-2: {robos_n_2_for_pred}")


--- Preparando datos de entrada para la predicción ---
Valores obtenidos: Mes a predecir: 12, Robos N-1: 5, Robos N-2: 1


In [23]:
print(f"\n--- Realizando predicción para Cuadrante 78, Mes {mes_a_predecir_specific} ---")

# Crear listas con un solo elemento para pasar a la función predict_next_month_robos
# La función espera listas de robos N-1, N-2 y cuadrantes, incluso para una sola predicción.
robos_n_menos_1_single = [robos_n_1_for_pred]
robos_n_menos_2_single = [robos_n_2_for_pred]
cuadrantes_specific = [78]

# Realizar la predicción usando la función ya definida
predicted_robos_specific = predict_next_month_robos(
    models,
    scaler_X,
    scaler_Y,
    mes_a_predecir_specific,
    robos_n_menos_1_single,
    robos_n_menos_2_single,
    cuadrantes_specific
)

# Mostrar el resultado
print(f"Predicción de robos para Cuadrante 78 en el Mes {mes_a_predecir_specific}: {predicted_robos_specific[78]}")


--- Realizando predicción para Cuadrante 78, Mes 12 ---
Predicción de robos para Cuadrante 78 en el Mes 12: 1


## Summary:

### Q&A
The predicted number of robberies for month 12 in quadrant 78 is 1.

### Data Analysis Key Findings
*   For Quadrant 78 and Month 11, the `Robos_N_1` value (representing robberies in the immediately previous month for prediction) was 5.
*   For Quadrant 78 and Month 10, the `Robos_N_2` value (representing robberies two months prior for prediction) was 1.
*   Using these historical values, the trained model predicted 1 robbery for Quadrant 78 in Month 12.

### Insights or Next Steps
*   The model successfully performed a single-point prediction for a specific quadrant and month, demonstrating its ability to forecast based on previous periods.
*   Further analysis could involve evaluating the model's performance on a broader range of quadrants and months, or comparing this prediction against historical averages for Quadrant 78 in Month 12 to assess its reasonableness.
